Naive Bayes Implementation

In [73]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [117]:
df = pd.read_csv('star_classification.csv')
df["class"].unique()
df = df[0:90000]
x = df.drop('class', axis=1)
x = x.drop("obj_ID", axis=1)
x = x.drop("spec_obj_ID", axis=1)
x = x.drop("rerun_ID", axis=1) 
y = df['class']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

classes = {'GALAXY':0, 'QSO':1, 'STAR':2 }
print(y.unique())
x

['GALAXY' 'QSO' 'STAR']


,alpha,delta,u,g,r,i,z,run_ID,cam_col,field_ID,redshift,plate,MJD,fiber_ID
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,2,79,0.634794,5812,56354,171
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,5,119,0.779136,10445,58158,427
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,2,120,0.644195,4576,55592,299
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,3,214,0.932346,9149,58039,775
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,3,137,0.116123,6121,56187,842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,207.063647,47.408481,24.83420,22.09308,20.42925,19.46181,19.21540,3650,3,107,0.537391,6749,56370,713
89996,196.658692,49.172683,19.07135,17.96796,17.35953,16.91640,16.66522,3650,3,60,0.054781,1280,52738,107
89997,190.575691,49.599248,24.64325,21.97179,21.13782,20.39246,19.67870,3650,3,33,0.754270,7415,57097,689
89998,208.030013,5.826776,19.79914,18.14168,17.24405,16.74151,16.37945,3841,1,287,0.082538,1805,53875,122


In [113]:
def transformYtoNum(y):
    # Transforms String classes to integers
    y_new = np.zeros((len(y)))
    for i in range(len(y)):
        y_new[i] = classes.get(y[i])

    return y_new

In [118]:
# Converting pd frames to numpy arrays
y_train = y_train.to_numpy()
x_train = x_train.to_numpy()
y_test = y_test.to_numpy()
x_test = x_test.to_numpy()

y_train = transformYtoNum(y_train)
y_test = transformYtoNum(y_test)

# prevent OVERFLOW in calculations
scaler=StandardScaler()
x_train_scaler=scaler.fit_transform(x_train)
x_test_scaler=scaler.fit_transform(x_test)

print(np.unique(y_train))

[0. 1. 2.]


In [119]:
def train_NB(x_train, y_train):

        means = dict()
        variances = dict()
        class_probs = dict()
        classes = np.unique(y_train)
        for c in classes:
           x_train_c = []
           for i in range(len(y_train)):
              if y_train[i] == c:
                x_train_c.append(x_train[i])

           x_train_c = np.array(x_train_c)
           means[c] = np.mean(x_train_c, axis=0)
           #print("Mean: \n", means)
           variances[c] = np.var(x_train_c, axis=0)
           #print("\nVariances :\n", variances, "\n")
           class_probs[c] = x_train_c.shape[0] / x_train.shape[0]
        
        return means, variances, class_probs

def calc_normal_density(x_test, mean, var):
        # Calculate the normal density for each attribute of the test data
        return np.exp(-((x_test - mean) ** 2) / (2 * var)) * 1 / np.sqrt(2 * np.pi * var)
        
        
        




def predict(x_test, means, variances, class_probs):
        
        predictions = []
        classes = [0,1,2]
        for x in x_test:
            classification_probs = {}
            for c in classes:
                densities_product = np.prod(calc_normal_density(x, means[c], variances[c])) # P(X|Ck)
                #print(densities_product)
                classification_probs[c] = class_probs[c] * densities_product # P(Ck|X) = P(X|Ck) * P(Ck)
                #print(classification_probs[c])
            
            predictions.append(max(classification_probs, key=classification_probs.get))
        return np.array(predictions)


          



means, variances, class_probs = train_NB(x_train_scaler, y_train)
predictions = predict(x_test_scaler, means, variances, class_probs)



correct = 0
for i in range(len(predictions)):
     if predictions[i] == y_test[i]:
          correct += 1

print ("Accuracy:", correct/len(y_test) * 100, "%")

Accuracy: 85.62592592592593 %
